In [1]:
from dataclasses import dataclass
from autogen_core import AgentId, MessageContext, RoutedAgent, message_handler
from autogen_core import SingleThreadedAgentRuntime
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.messages import TextMessage
from autogen_ext.models.openai import OpenAIChatCompletionClient
from dotenv import load_dotenv
from autogen_core import SingleThreadedAgentRuntime

load_dotenv(override=True)

True

In [2]:
@dataclass
class Message:
    content: str

In [13]:
class Player1Agent(RoutedAgent):
    def __init__(self, name:str):
        super().__init__(name)
        model_client = OpenAIChatCompletionClient(model='gpt-4o-mini', temperature=1.0)
        self._delegate = AssistantAgent(name,  model_client=model_client)

    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        text_message = TextMessage(content=message.content, source="user")
        response = await self._delegate.on_messages([text_message], ctx.cancellation_token)
        return Message(content=response.chat_message.content)
    

class Player2Agent(RoutedAgent):
    def __init__(self, name:str):
        super().__init__(name)
        model_client = OpenAIChatCompletionClient(model='llama3.2', temperature=1.0)
        self._delegate = AssistantAgent(name, model_client=model_client)

    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        text_message = TextMessage(content=message.content, source="user")
        response = await self._delegate.on_messages([text_message], ctx.cancellation_token)
        return Message(content=response.chat_message.content)

In [17]:
JUDGE = "You are judging a game of rock, paper, scissors. The players have made these choices:\n"

class RockPaperScissorAgent(RoutedAgent):
    def __init__(self, name:str):
        super().__init__(name)
        model_client = OpenAIChatCompletionClient(model='gpt-4o-mini', temperature=1.0)
        self._delegate = AssistantAgent(name, model_client=model_client)

    @message_handler
    async def handle_my_message_type(self, message:Message, ctx: MessageContext) -> Message:
        instruction = "You are playing rock, paper, scissors. Respond only with the one word, one of the following: rock, paper, or scissors."
        message = Message(content=instruction)
        inner_1 = AgentId("player1", "default")
        inner_2 = AgentId("player2", "default")
        inner_3 = AgentId("player3", "default")
        response1 = await self.send_message(message, inner_1)
        response2 = await self.send_message(message, inner_2)
        response3 = await self.send_message(message, inner_3)
        result = f"Player 1: {response1.content}\nPlayer 2: {response2.content}\nPlayer 3: {response3.content}\n"
        judgement = f"{JUDGE}{result}Who wins?"
        message = TextMessage(content=judgement, source="user")
        response = await self._delegate.on_messages([message], ctx.cancellation_token)
        return Message(content=result + response.chat_message.content)

In [18]:
runtime = SingleThreadedAgentRuntime()
await Player1Agent.register(runtime,"player1",lambda:Player1Agent('player1'))
await Player1Agent.register(runtime,"player2",lambda:Player1Agent('player2'))
await Player1Agent.register(runtime,"player3",lambda:Player1Agent('player3'))
await RockPaperScissorAgent.register(runtime, "rock_paper_scissors", lambda: RockPaperScissorAgent('rock_paper_scissors'))
runtime.start()

In [20]:
agent_id = AgentId("rock_paper_scissors", "default")
message = Message(content="Go")
response = await runtime.send_message(message, agent_id)
print(response.content)

Player 1: paper
Player 2: scissors
Player 3: rock
In this game of rock, paper, scissors:
- Paper beats rock.
- Scissors beat paper.
- Rock beats scissors.

Now, let's analyze the choices:
- Player 1 (paper) is beaten by Player 2 (scissors).
- Player 2 (scissors) beats Player 1 (paper).
- Player 3 (rock) is beaten by Player 2 (scissors).

Therefore, Player 2 wins, as their choice of scissors beats Player 1's paper. Player 3 doesn't have any wins against Player 2 either.

TERMINATE


In [21]:
await runtime.stop()
await runtime.close()